# Milvus

>[Milvus](https://milvus.io/docs/overview.md) 是一个数据库，用于存储、索引和管理由深度神经网络和其他机器学习 (ML) 模型生成的海量 embedding 向量。

本 Notebook 展示了如何使用与 Milvus 向量数据库相关的功能。

## 设置

您需要安装 `langchain-milvus`，命令为 `pip install -qU langchain-milvus`，才能使用此集成。

In [1]:
pip install -qU langchain_milvus

Note: you may need to restart the kernel to use updated packages.


### 凭证

使用 `Milvus` 向量库不需要任何凭证。

## 初始化

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>

In [2]:
# | output: false
# | echo: false
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

### Milvus Lite

最简单的原型设计方法是使用 Milvus Lite，所有数据都存储在本地向量数据库文件中。只能使用 Flat 索引。

In [3]:
from langchain_milvus import Milvus

URI = "./milvus_example.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)

### Milvus 服务器

如果您拥有大量数据（例如，超过一百万个向量），我们建议在 [Docker](https://milvus.io/docs/install_standalone-docker.md#Start-Milvus) 或 [Kubernetes](https://milvus.io/docs/install_cluster-milvusoperator.md) 上设置一个性能更高的 Milvus 服务器。

Milvus 服务器支持多种[索引](https://milvus.io/docs/index.md?tab=floating)。利用这些不同的索引可以显著增强检索能力并加快检索过程，以满足您的特定需求。

例如，以 Milvus Standalone 为例。要启动 Docker 容器，您可以运行以下命令：

In [ ]:
!curl -sfL https://raw.githubusercontent.com/milvus-io/milvus/master/scripts/standalone_embed.sh -o standalone_embed.sh

!bash standalone_embed.sh start

Password:

在此，我们创建一个 Milvus 数据库：

In [15]:
from pymilvus import Collection, MilvusException, connections, db, utility

conn = connections.connect(host="127.0.0.1", port=19530)

# Check if the database exists
db_name = "milvus_demo"
try:
    existing_databases = db.list_database()
    if db_name in existing_databases:
        print(f"Database '{db_name}' already exists.")

        # Use the database context
        db.using_database(db_name)

        # Drop all collections in the database
        collections = utility.list_collections()
        for collection_name in collections:
            collection = Collection(name=collection_name)
            collection.drop()
            print(f"Collection '{collection_name}' has been dropped.")

        db.drop_database(db_name)
        print(f"Database '{db_name}' has been deleted.")
    else:
        print(f"Database '{db_name}' does not exist.")
        database = db.create_database(db_name)
        print(f"Database '{db_name}' created successfully.")
except MilvusException as e:
    print(f"An error occurred: {e}")

Database 'milvus_demo' does not exist.
Database 'milvus_demo' created successfully.


请注意下面 URI 的变化。实例初始化后，请导航至 http://127.0.0.1:9091/webui 查看本地 Web UI。

以下是使用 Milvus 数据库服务创建向量存储实例的示例：

In [16]:
from langchain_milvus import BM25BuiltInFunction, Milvus

URI = "http://localhost:19530"

vectorstore = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI, "token": "root:Milvus", "db_name": "milvus_demo"},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    consistency_level="Strong",
    drop_old=False,  # set to True if seeking to drop the collection with that name if it exists
)

> 如果您想使用 Zilliz Cloud，即 Milvus 的全托管云服务，请调整 `uri` 和 `token`，它们分别对应 Zilliz Cloud 中的 [公共端点](https://docs.zilliz.com/docs/byoc/quick-start#free-cluster-details) 和 [API 密钥](https://docs.zilliz.com/docs/byoc/quick-start#free-cluster-details)。

### 使用 Milvus Collection 对数据进行分区

您可以在同一个 Milvus 实例中将不相关的文档存储在不同的 collection 中。

以下是如何创建一个新的 collection：

In [ ]:
from langchain_core.documents import Document

vector_store_saved = Milvus.from_documents(
    [Document(page_content="foo!")],
    embeddings,
    collection_name="langchain_example",
    connection_args={"uri": URI},
)

以下是检索已存储集合的方法：

In [ ]:
vector_store_loaded = Milvus(
    embeddings,
    connection_args={"uri": URI},
    collection_name="langchain_example",
)

## 管理向量库

创建向量库后，我们可以通过添加和删除不同的项目与之交互。

### 向向量库添加项目

我们可以使用 `add_documents` 函数向向量库添加项目。

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

### 从向量存储中删除项目

In [ ]:
vector_store.delete(ids=[uuids[-1]])

(insert count: 0, delete count: 1, upsert count: 0, timestamp: 0, success count: 0, err count: 0, cost: 0)

## 查询向量数据库

一旦您的向量数据库创建完成并且已添加了相关文档，您很可能希望在链或代理运行时对其进行查询。

### 直接查询

#### 相似性搜索

可以按如下方式使用元数据过滤执行简单的相似性搜索：

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    expr='source == "tweet"',
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'pk': '9905001c-a4a3-455e-ab94-72d0ed11b476', 'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'pk': '1206d237-ee3a-484f-baf2-b5ac38eeb314', 'source': 'tweet'}]


#### 相似性搜索（带分数）

您也可以按照分数进行搜索：

In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, expr='source == "news"'
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=21192.628906] bar [{'pk': '2', 'source': 'https://example.com'}]


要了解使用 `Milvus` 向量库的所有可用搜索选项的完整列表，您可以访问 [API 参考](https://python.langchain.com/api_reference/milvus/vectorstores/langchain_milvus.vectorstores.milvus.Milvus.html)。

### 通过转换为 Retriever 进行查询

您还可以将向量库转换为 Retriever，以便在您的链中更轻松地使用。

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(metadata={'pk': 'eacc7256-d7fa-4036-b1f7-83d7a4bee0c5', 'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

## 混合搜索

最常见的混合搜索场景是密集+稀疏的混合搜索，即使用语义向量相似性和精确的关键字匹配来检索候选结果。这两种方法的检索结果会被合并、重新排序，然后传递给 LLM 以生成最终答案。这种方法在精度和语义理解之间取得了平衡，对于各种查询场景都非常有效。

### 全文搜索
自 [Milvus 2.5](https://milvus.io/blog/introduce-milvus-2-5-full-text-search-powerful-metadata-filtering-and-more.md) 起，通过稀疏 BM25 方法原生支持全文搜索，即将 BM25 算法表示为稀疏向量。Milvus 接受原始文本作为输入，并自动将其转换为存储在指定字段中的稀疏向量，无需手动生成稀疏嵌入。

对于全文搜索，Milvus VectorStore 接受一个 `builtin_function` 参数。通过此参数，您可以传入 `BM25BuiltInFunction` 的实例。这与通常将密集嵌入传递给 `VectorStore` 的语义搜索不同，

以下是一个在 Milvus 中结合 OpenAI 密集嵌入进行语义搜索和 BM25 进行全文搜索的混合搜索简单示例：

In [4]:
from langchain_milvus import BM25BuiltInFunction, Milvus
from langchain_openai import OpenAIEmbeddings

vectorstore = Milvus.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings(),
    builtin_function=BM25BuiltInFunction(),
    # `dense` is for OpenAI embeddings, `sparse` is the output field of BM25 function
    vector_field=["dense", "sparse"],
    connection_args={
        "uri": URI,
    },
    consistency_level="Strong",
    drop_old=True,
)

> - 使用 `BM25BuiltInFunction` 时，请注意，Milvus Standalone 和 Milvus Distributed 版本提供全文搜索功能，而 Milvus Lite 不支持（尽管该功能已列入未来规划）。它也将很快在 Zilliz Cloud（全托管 Milvus）中提供。如需更多信息，请联系 support@zilliz.com。

在上面的代码中，我们定义了 `BM25BuiltInFunction` 的一个实例，并将其传递给 `Milvus` 对象。`BM25BuiltInFunction` 是 Milvus 中 [`Function`](https://milvus.io/docs/manage-collections.md#Function) 的一个轻量级包装类。我们可以使用它配合 `OpenAIEmbeddings` 来初始化一个密集向量 + 稀疏向量混合搜索的 Milvus 向量存储实例。

`BM25BuiltInFunction` 不需要客户端传递语料或进行训练，所有这些都会在 Milvus 服务器端自动处理，因此用户无需关心任何词汇和语料。此外，用户还可以自定义 [analyzer](https://milvus.io/docs/analyzer-overview.md#Analyzer-Overview) 来在 BM25 中实现自定义文本处理。

### 对候选集进行重新排序

在检索的第一阶段之后，我们需要对候选集进行重新排序以获得更好的结果。您可以参考 [重新排序](https://milvus.io/docs/reranking.md#Reranking) 以获取更多信息。

以下是一个加权重排的示例：

In [ ]:
query = "What are the novels Lila has written and what are their contents?"

vectorstore.similarity_search(
    query, k=1, ranker_type="weighted", ranker_params={"weights": [0.6, 0.4]}
)

## 用于检索增强生成的使用方法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程](/docs/tutorials/)
- [操作指南：使用 RAG 进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/retrieval)

### 按用户检索

在构建检索应用时，通常需要考虑多个用户。这意味着您可能存储的数据不仅限于一个用户，而是多个不同用户的数据，并且这些用户之间应该无法看到彼此的数据。

Milvus 建议使用 [partition_key](https://milvus.io/docs/multi_tenancy.md#Partition-key-based-multi-tenancy) 来实现多租户。以下是一个示例：
> Partition key 功能在 Milvus Lite 中不可用，如果您想使用它，需要启动 Milvus 服务器，如上所述。

In [ ]:
from langchain_core.documents import Document

docs = [
    Document(page_content="i worked at kensho", metadata={"namespace": "harrison"}),
    Document(page_content="i worked at facebook", metadata={"namespace": "ankush"}),
]
vectorstore = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"uri": URI},
    drop_old=True,
    partition_key_field="namespace",  # Use the "namespace" field as the partition key
)

为了使用分区键进行搜索，您应该在搜索请求的布尔表达式中包含以下任一项：

`search_kwargs={"expr": '<partition_key> == "xxxx"'}`

`search_kwargs={"expr": '<partition_key> == in ["xxx", "xxx"]'}`

请将 `<partition_key>` 替换为指定为分区键的字段名称。

Milvus 根据指定的分区键进行分区更改，根据分区键过滤实体，并在过滤后的实体中进行搜索。

In [ ]:
# This will only get documents for Ankush
vectorstore.as_retriever(search_kwargs={"expr": 'namespace == "ankush"'}).invoke(
    "where did i work?"
)

[Document(page_content='i worked at facebook', metadata={'namespace': 'ankush'})]

In [ ]:
# This will only get documents for Harrison
vectorstore.as_retriever(search_kwargs={"expr": 'namespace == "harrison"'}).invoke(
    "where did i work?"
)

[Document(page_content='i worked at kensho', metadata={'namespace': 'harrison'})]

## API 参考

如需了解 __ModuleName__VectorStore 所有功能和配置的详细文档，请前往 API 参考：https://python.langchain.com/api_reference/milvus/vectorstores/langchain_milvus.vectorstores.milvus.Milvus.html